In [13]:
import os as os
import numpy as np
import pandas as pd
import polars as pl
import tqdm
import slippi as slp
from joblib import Parallel, delayed
from multiprocessing import Manager

<h2> Preliminary Functions </h2>

We use these functions to one-hot encode the button bitmask and get the frame data for a given port number and frames object.

In [14]:
# Set the number of time steps in the model inputs
frames_per_input = 60 * 12  # 12 seconds of gameplay

# Function to one-hot encode controller bitmask
def one_hot_encode(bitmask):
    labels = ['DPAD_LEFT', 'DPAD_RIGHT', 'DPAD_DOWN', 'DPAD_UP', 'Z', 'R', 'L', 'A', 'B', 'X', 'Y', 'START']
    encoded_values = [1, 2, 4, 8, 16, 32, 64, 256, 512, 1024, 2048, 4096]

    # Create a dictionary mapping labels to their encoded values
    label_to_value = dict(zip(labels, encoded_values))

    # Initialize a list to store the one-hot encoded values
    one_hot_encoded = [0] * len(labels)

    # Iterate through labels and set the corresponding one-hot encoded value
    for label, value in label_to_value.items():
        if bitmask & value:
            one_hot_encoded[labels.index(label)] = 1

    return one_hot_encoded

# Function to get frame data for a given set of frames and port
def get_frame_inputs(frames, port):
    sheik_inputs = np.empty((len(frames)-300, 18))  # Initialize an empty NumPy array    
    for i, frame in enumerate(frames[300: ]):   # Takes all the frames, skipping the first 5 seconds.
        buttons = one_hot_encode(frame.ports[port].leader.pre.buttons.physical.value)
        j_x = frame.ports[port].leader.pre.joystick.x
        j_y = frame.ports[port].leader.pre.joystick.y
        c_x = frame.ports[port].leader.pre.cstick.x
        c_y = frame.ports[port].leader.pre.cstick.y
        t_l = frame.ports[port].leader.pre.triggers.physical.l
        t_r = frame.ports[port].leader.pre.triggers.physical.r

        frame_data = buttons + [j_x, j_y, c_x, c_y, t_l, t_r]
        sheik_inputs[i] = frame_data

    return sheik_inputs





In [15]:
# # Function to process a single SLP file and append to shared lists
# def process_slp_file(slp_file, dataset_path, time_series_list, label_list, ids):
#     try:
#         file_path = os.path.join(dataset_path, slp_file)
#         game = slp.Game(file_path)
#         frames = game.frames

#         if len(frames) < 300 + frames_per_input:  # Ignore games that are <3600 frames (i.e. <60 seconds)
#             return

#         # List occupied ports
#         occupied_ports = [i for i, port in enumerate(game.start.players) if port is not None]
#         port_1 = occupied_ports[0]
#         port_2 = occupied_ports[1]

#         if len(occupied_ports) > 2:  # Ignore games that aren't singles
#             return

#         # Determine characters playing
#         port_1_character = game.start.players[port_1].character.name
#         port_2_character = game.start.players[port_2].character.name

#         frame_data = get_frame_data(frames, port_1)
#         time_series_list.append(frame_data)
#         label_list.append(1 if port_1_character == 'SHEIK' else 0)
#         ids.append(slp_file)
#         frame_data = get_frame_data(frames, port_2)
#         time_series_list.append(frame_data)
#         label_list.append(1 if port_2_character == 'SHEIK' else 0)
#         ids.append(slp_file)
#     except Exception as e:
#         print(f"Error processing {slp_file}: {str(e)}")

In [16]:
        
# Set your dataset_path and frames_per_input
# dataset_path = './Slippi_Public_Dataset_v3/'

# slp_files = [file for file in os.listdir(dataset_path)]
# # print(len(slp_files))

# # Create polars data frame to store results
# game_data_df = pl.DataFrame(
#     {
#         "TimeSeries": [pl.col(pl.col([1, 2, 3])), pl.col(pl.col([4, 5, 6]))],
#         "Label": [pl.col(0), pl.col(1)],
#         "FName": [pl.col("file1.slp"), pl.col("file2.slp")]
#     }
# )

# Use joblib to parallelize processing of SLP files
# Parallel(n_jobs=-1, verbose=10)(delayed(process_slp_file)(slp_file, dataset_path, time_series_list, label_list, ids) for slp_file in tqdm.tqdm(slp_files))

# Create a DataFrame from the results
# df = pd.DataFrame({"TimeSeries": list(time_series_list), "Label": list(label_list), "FName": list(ids)})


# df.sort_values(by=['FName','Label'],inplace=True)
# df.reset_index(drop=True)
# print(df)

In [22]:
# Set the number of time steps in the model inputs
frames_per_input = 60 * 12  # 12 seconds of gameplay

dataset_path = './Slippi_Public_Dataset_v3/'

# List of file names
slp_files = [file for file in os.listdir(dataset_path)]

slp_file = slp_files[10]

# A single file path
file_path = os.path.join(dataset_path, slp_file)


# game_data_df = 

# Function to process a single SLP file and append to shared polars df
def process_slp_file(slp_file, dataset_path,game_data_df,schema):#,df):#, time_series_list, label_list, ids):
    try:
        file_path = os.path.join(dataset_path, slp_file)
        game = slp.Game(file_path)
        frames = game.frames

        if len(frames) < 300 + frames_per_input:  # Ignore games that are <3600 frames (i.e. <60 seconds)
            return

        # List occupied ports
        occupied_ports = [i for i, port in enumerate(game.start.players) if port is not None]
        port_1 = occupied_ports[0]
        port_2 = occupied_ports[1]

        if len(occupied_ports) > 2:  # Ignore games that aren't singles
            return

        port_1_frame_inputs = get_frame_inputs(frames, port_1)
        port_2_frame_inputs = get_frame_inputs(frames, port_2)
        
        game_data_dict = {
            # Game Data
            'file': slp_file,
            'stage_name': game.start.stage.name,
            'stage_value': game.start.stage.value,
            'is_pal': game.start.is_pal,
            'is_frozen_ps': game.start.is_frozen_ps,
            # Player 1 Data
            'port_1': port_1,
            'port_1_character_name': game.start.players[port_1].character.name,
            'port_1_character_value': game.start.players[port_1].character.value,
            'port_1_type_name': game.start.players[port_1].type.name,
            'port_1_type_value': game.start.players[port_1].type.value,
            'port_1_stocks': game.start.players[port_1].stocks,
            'port_1_costume': game.start.players[port_1].costume,
            'port_1_ucf_dash_back_name': game.start.players[port_1].ucf.dash_back.name,
            'port_1_ucf_dash_back_value': game.start.players[port_1].ucf.dash_back.value,
            'port_1_ucf_shield_drop_name': game.start.players[port_1].ucf.shield_drop.name,
            'port_1_ucf_shield_drop_value': game.start.players[port_1].ucf.shield_drop.value,
            'port_1_tag': game.start.players[port_1].tag,
            # Player 2 Data
            'port_2': port_2,
            'port_2_character_name': game.start.players[port_2].character.name,
            'port_2_character_value': game.start.players[port_2].character.value,
            'port_2_type_name': game.start.players[port_2].type.name,
            'port_2_type_value': game.start.players[port_2].type.value,
            'port_2_stocks': game.start.players[port_2].stocks,
            'port_2_costume': game.start.players[port_2].costume,
            'port_2_ucf_dash_back_name': game.start.players[port_2].ucf.dash_back.name,
            'port_2_ucf_dash_back_value': game.start.players[port_2].ucf.dash_back.value,
            'port_2_ucf_shield_drop_name': game.start.players[port_2].ucf.shield_drop.name,
            'port_2_ucf_shield_drop_value': game.start.players[port_2].ucf.shield_drop.value,
            'port_2_tag': game.start.players[port_2].tag,
            # Player 1 Inputs
            'port_1_DPAD_LEFT': [port_1_frame_inputs[:, 0]],
            'port_1_DPAD_RIGHT': [port_1_frame_inputs[:, 1]],
            'port_1_DPAD_DOWN': [port_1_frame_inputs[:, 2]],
            'port_1_DPAD_UP': [port_1_frame_inputs[:, 3]],
            'port_1_Z': [port_1_frame_inputs[:, 4]],
            'port_1_R': [port_1_frame_inputs[:, 5]],
            'port_1_L': [port_1_frame_inputs[:, 6]],
            'port_1_A': [port_1_frame_inputs[:, 7]],
            'port_1_B': [port_1_frame_inputs[:, 8]],
            'port_1_X': [port_1_frame_inputs[:, 9]],
            'port_1_Y': [port_1_frame_inputs[:, 10]],
            'port_1_START': [port_1_frame_inputs[:, 11]],
            'port_1_J_X': [port_1_frame_inputs[:, 12]],
            'port_1_J_Y': [port_1_frame_inputs[:, 13]],
            'port_1_C_X': [port_1_frame_inputs[:, 14]],
            'port_1_C_Y': [port_1_frame_inputs[:, 15]],
            'port_1_T_L': [port_1_frame_inputs[:, 16]],
            'port_1_T_R': [port_1_frame_inputs[:, 17]],
            # Player 2 Inputs
            'port_2_DPAD_LEFT': [port_2_frame_inputs[:, 0]],
            'port_2_DPAD_RIGHT': [port_2_frame_inputs[:, 1]],
            'port_2_DPAD_DOWN': [port_2_frame_inputs[:, 2]],
            'port_2_DPAD_UP': [port_2_frame_inputs[:, 3]],
            'port_2_Z': [port_2_frame_inputs[:, 4]],
            'port_2_R': [port_2_frame_inputs[:, 5]],
            'port_2_L': [port_2_frame_inputs[:, 6]],
            'port_2_A': [port_2_frame_inputs[:, 7]],
            'port_2_B': [port_2_frame_inputs[:, 8]],
            'port_2_X': [port_2_frame_inputs[:, 9]],
            'port_2_Y': [port_2_frame_inputs[:, 10]],
            'port_2_START': [port_2_frame_inputs[:, 11]],
            'port_2_J_X': [port_2_frame_inputs[:, 12]],
            'port_2_J_Y': [port_2_frame_inputs[:, 13]],
            'port_2_C_X': [port_2_frame_inputs[:, 14]],
            'port_2_C_Y': [port_2_frame_inputs[:, 15]],
            'port_2_T_L': [port_2_frame_inputs[:, 16]],
            'port_2_T_R': [port_2_frame_inputs[:, 17]],
        }
    
        # game_data_dict['port_1_inputs'] = [get_frame_inputs(frames, port_1)]
        # game_data_dict['port_2_inputs'] = [get_frame_inputs(frames, port_2)]
        # Convert the game_data_dict to a Polars DataFrame without port_1_inputs and port_2_inputs


        return game_data_dict
    except Exception as e:
        print(f"Error processing {slp_file}: {str(e)}")
        


    
# Create an empty Polars DataFrame with the desired schema
schema =    [
    ('file', pl.String),
    ('stage_name', pl.String),
    ('stage_value', pl.Int8),
    ('is_pal', pl.Boolean),
    ('is_frozen_ps', pl.Boolean),
    # Player 1 Data
    ('port_1', pl.Int32),
    ('port_1_character_name', pl.String),
    ('port_1_character_value', pl.Int32),
    ('port_1_type_name', pl.String),
    ('port_1_type_value', pl.Boolean),
    ('port_1_stocks', pl.Int32),
    ('port_1_costume', pl.Int32),
    ('port_1_ucf_dash_back_name', pl.String),
    ('port_1_ucf_dash_back_value', pl.Int32),
    ('port_1_ucf_shield_drop_name', pl.String),
    ('port_1_ucf_shield_drop_value', pl.Int32),
    ('port_1_tag', pl.String),
    # Player 2 Data
    ('port_2', pl.Int32),
    ('port_2_character_name', pl.String),
    ('port_2_character_value', pl.Int32),
    ('port_2_type_name', pl.String),
    ('port_2_type_value', pl.Boolean),
    ('port_2_stocks', pl.Int32),
    ('port_2_costume', pl.Int32),
    ('port_2_ucf_dash_back_name', pl.String),
    ('port_2_ucf_dash_back_value', pl.Int32),
    ('port_2_ucf_shield_drop_name', pl.String),
    ('port_2_ucf_shield_drop_value', pl.Int32),
    ('port_2_tag', pl.String),
    # Input Data
    # ('port_1_inputs', pl.Object),
    # ('port_2_inputs', pl.Object)
        # Input Data for Player 1
    ('port_1_DPAD_LEFT', pl.Object),
    ('port_1_DPAD_RIGHT', pl.Object),
    ('port_1_DPAD_DOWN', pl.Object),
    ('port_1_DPAD_UP', pl.Object),
    ('port_1_Z', pl.Object),
    ('port_1_R', pl.Object),
    ('port_1_L', pl.Object),
    ('port_1_A', pl.Object),
    ('port_1_B', pl.Object),
    ('port_1_X', pl.Object),
    ('port_1_Y', pl.Object),
    ('port_1_START', pl.Object),
    ('port_1_J_X', pl.Object),
    ('port_1_J_Y', pl.Object),
    ('port_1_C_X', pl.Object),
    ('port_1_C_Y', pl.Object),
    ('port_1_T_L', pl.Object),
    ('port_1_T_R', pl.Object),
    # Input Data for Player 2
    ('port_2_DPAD_LEFT', pl.Object),
    ('port_2_DPAD_RIGHT', pl.Object),
    ('port_2_DPAD_DOWN', pl.Object),
    ('port_2_DPAD_UP', pl.Object),
    ('port_2_Z', pl.Object),
    ('port_2_R', pl.Object),
    ('port_2_L', pl.Object),
    ('port_2_A', pl.Object),
    ('port_2_B', pl.Object),
    ('port_2_X', pl.Object),
    ('port_2_Y', pl.Object),
    ('port_2_START', pl.Object),
    ('port_2_J_X', pl.Object),
    ('port_2_J_Y', pl.Object),
    ('port_2_C_X', pl.Object),
    ('port_2_C_Y', pl.Object),
    ('port_2_T_L', pl.Object),
    ('port_2_T_R', pl.Object),
    
]



game_data_df = pl.DataFrame([], schema=schema)

print(process_slp_file(slp_file, dataset_path,game_data_df,schema))
print(game_data_df)

# print(game_data_df)

# game_data_df

{'file': '00_37_01.564Z [314] Fox + Sheik (FoD).slp', 'stage_name': 'FOUNTAIN_OF_DREAMS', 'stage_value': 2, 'is_pal': False, 'is_frozen_ps': True, 'port_1': 0, 'port_1_character_name': 'FOX', 'port_1_character_value': 2, 'port_1_type_name': 'HUMAN', 'port_1_type_value': 0, 'port_1_stocks': 4, 'port_1_costume': 1, 'port_1_ucf_dash_back_name': 'UCF', 'port_1_ucf_dash_back_value': 1, 'port_1_ucf_shield_drop_name': 'UCF', 'port_1_ucf_shield_drop_value': 1, 'port_1_tag': '３１４', 'port_2': 1, 'port_2_character_name': 'SHEIK', 'port_2_character_value': 19, 'port_2_type_name': 'HUMAN', 'port_2_type_value': 0, 'port_2_stocks': 4, 'port_2_costume': 3, 'port_2_ucf_dash_back_name': 'UCF', 'port_2_ucf_dash_back_value': 1, 'port_2_ucf_shield_drop_name': 'UCF', 'port_2_ucf_shield_drop_value': 1, 'port_2_tag': '', 'port_1_DPAD_LEFT': [array([0., 0., 0., ..., 0., 0., 0.])], 'port_1_DPAD_RIGHT': [array([0., 0., 0., ..., 0., 0., 0.])], 'port_1_DPAD_DOWN': [array([0., 0., 0., ..., 0., 0., 0.])], 'port_1_DP

In [18]:
import polars as pl

# Create an example two-dimensional array
two_dimensional_array = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
]

# Create a Polars DataFrame with the two-dimensional array in one cell
data = {'two_dimensional_data': [two_dimensional_array]}
schema = [('two_dimensional_data', pl.Object)]
df = pl.DataFrame(data, schema=schema)

# Show the DataFrame
print(df)

shape: (1, 1)
┌───────────────────────────────────┐
│ two_dimensional_data              │
│ ---                               │
│ object                            │
╞═══════════════════════════════════╡
│ [[1, 2, 3], [4, 5, 6], [7, 8, 9]… │
└───────────────────────────────────┘


In [19]:
import polars as pl
import numpy as np

# Create an example 2D NumPy array
two_dimensional_array = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
])

# Create a Polars DataFrame with the 2D NumPy array in one cell
data = {'two_dimensional_data': [two_dimensional_array]}
schema = [('two_dimensional_data', pl.Object)]
df = pl.DataFrame(data, schema=schema)

# Show the DataFrame
print(df)


shape: (1, 1)
┌──────────────────────┐
│ two_dimensional_data │
│ ---                  │
│ object               │
╞══════════════════════╡
│ [[1 2 3]             │
│  [4 5 6]             │
│  [7 8 9]]            │
└──────────────────────┘
